# Extract Discharge Diagnosis Subsections
This notebook queries an existing MIMIC-III replica database and extracts 'Discharge Diagnosis' subsections of discharge summary reports

In [1]:
import pandas as pd
from extract import append_addendums, mimic3_notes, mimic3_diagnosis_icd
import os
import re

In [ ]:
# ADD YOUR MIMIC-III Database host, dbname, username and password
username = ''
password = ''
host = '10.200.102.52' # hostname
dbname = 'mimic'

In [ ]:
# these should write somewhere ... 
conn = psycopg2.connect(f'host={host} dbname={dbname} user={username} password={password}')
codes = mimic3_diagnosis_icd(conn) # if successful, will write to data/admission_codes.csv
notes = mimic3_notes(conn) # if successful, will write to data/notes.csv

In [8]:
notes = append_addendums(notes)
codes = pd.read_csv('data/admission_codes.csv')
notes = pd.read_csv('data/notes.csv')

In [9]:
notes_out = notes.loc[:, ['hadm_id', 'text']]
notes_out['name'] = notes_out['hadm_id'].apply(lambda i: f'Admission ID:{str(i)}')
notes_out = notes_out.drop('hadm_id', axis=1).loc[:, ['name', 'text']]
notes_out.to_csv('mimic3_full_notes.csv', index=False)

In [9]:
top_50_codes = pd.DataFrame(codes.groupby('icd9_code').count().sort_values('hadm_id', ascending=False)['hadm_id'][0:50])
top_50_codes.reset_index(inplace=True)
top_50_codes['desc'] = top_50_codes.apply(lambda x: codes[codes['icd9_code'] == x.icd9_code].short_title.iloc[0], axis=1)
top_50_codes.columns = ['ICD9_Code', 'Count', 'Desc']
top_50_codes.head(20)

In [12]:
DISCHARGE_DIAG_SYNONYMS = [
    'discharge diagnosis',
    'discharge diagnoses',
    'discharge diagnose',
    'final diagnosis',
    'final diagnoses',
    'death diagnosis',
    'death diagnoses',
    'discharge diagnosis list',
    'discharge diagnoses list',
    'death diagnosis',
]

In [13]:
srs = notes.text
srs_hadmid = notes.hadm_id

In [14]:
diag_pattern = '|'.join([f'(?<={term}:)(?: ?\n?\n?.*)(?:.+\n)+' for term in DISCHARGE_DIAG_SYNONYMS])

In [15]:
reg = re.compile(f'({diag_pattern})', re.M | re.I)

In [16]:
# for non matches, that are excluded Series.str.extractAll
single_extract = srs.str.extract(reg)
no_matches = single_extract.loc[single_extract.isnull()[0], :]

In [17]:
all_matches = srs.str.extractall(reg)

In [18]:
groups = list(all_matches[0:100].reset_index().groupby('level_0'))

In [19]:
# tuples of: index of hadm_id,  last match in the discharge_summary
last_matches = [(g.iloc[-1, :].level_0, g.iloc[-1, :][0]) for k, g in all_matches.reset_index().groupby('level_0')]

In [21]:
no_match_tuples =[(d[0], d[1][0]) for d in single_extract[pd.isna(single_extract.reset_index()[0])].iterrows()]

In [23]:
last_matches.extend(no_match_tuples)

In [24]:
sorted_matches = sorted(last_matches, key=lambda x: x[0])

In [25]:
matches = pd.DataFrame(sorted_matches, columns=['idx', 'match'])

In [26]:
matches = matches.drop('idx', axis=1)
matches['note'] = srs
matches['hadm_id'] = srs_hadmid
matches.reset_index(drop=True, inplace=True)

### The matches for diagnosis: , diagnoses: can result in fale positives on sections that are not the final diagnosis. These are marked as false positives.

In [28]:
matches['dd_line_len'] = matches.match.str.split('\n').apply(lambda x: len(x) if type(x) == list else 1)

In [29]:
# arbitrary cutoff, DD subsetions longer than 60 lines are excluded.
no_dis_diag = matches[(pd.isna(matches.match)) | (matches['dd_line_len'] > 60)]
dis_diag = matches[(~pd.isna(matches.match)) & (matches['dd_line_len'] <= 60)]

- death of patient: 50
- followed up with primary care dr: 51, 55, 58 (medications on discharge), 
- followed up with non-icu department in hospital: 56, 101960, 101967, 101969
- patient is still in hospital at time of writing:  53
- notes are incomplete: 54, 57 (addendum referred to but not provided in dataset), 61
- addenudems are added, but no clearly defined list still: 101936

In [ ]:
# should be possible to further clean this to just include the hospital course section..

In [ ]:
[print(f'{r.hadm_id}\n{r.match}') for r in dis_diag.loc[181:190, ['match', 'hadm_id']].itertuples()]

In [ ]:
matches[matches.match.str.contains('GERD').fillna(False)]

### Total Notes with a discharge diag section: 

In [475]:
print(f'Summaries w/ DD Sections:{dis_diag.shape[0]}')
print(f'Summaries w/o DD Sections:{no_dis_diag.shape[0]}')
print(f'{round(dis_diag.shape[0] / (dis_diag.shape[0] + no_dis_diag.shape[0]) * 100, 2)}%')

Summaries w/ DD Sections:48898
Summaries w/o DD Sections:3828
92.74%


## Convert dis_diag to MedCATTrainer Upload Format
This is a two column flat table, wit column headers, (name, text).

In [444]:
medcat_df = dis_diag.loc[:, ['hadm_id', 'match']]
medcat_df['name'] = medcat_df['hadm_id'].apply(lambda i: f'Admission ID:{str(i)}')
medcat_df = medcat_df.drop('hadm_id', axis=1)

In [462]:
medcat_df.columns = ['text', 'name']

In [493]:
medcat_df = medcat_df.loc[:, ['name', 'text']]

In [497]:
medcat_df.to_csv('mimic_dd_icd9_sections.csv', index=False)